In [84]:
import sqlite3
import collections
import json

processing kanjis

In [157]:
class Kanji:
    def __init__(self, kanji, reading):
        self.kanji = kanji
        self.reading = reading

kanji_dictionary = dict()

def process_bank(bank):
    kanji_dict = json.load(bank)
    for kanji in kanji_dict:
        surface =  kanji[0]
        reading = kanji[1]
        kanji_dictionary[surface] = Kanji(surface,reading.replace(" ", "/"))
process_bank(open("kanji_bank_1.json"))
process_bank(open("kanji_bank_2.json"))

In [158]:
kanji_dictionary["㒵"].reading

'mạc/mạo/mộc'

process dictionary

In [166]:
file = open('star_nhatviet.tab','r')
words = file.read().split('\n')

class DictEntry:
    def __init__(self,word,kanji_reading, kanjis, reading, desc):
        self.word = word
        self.kanji_reading = kanji_reading
        self.kanjis = kanjis
        self.reading = reading
        self.desc = desc

def look_up_kanji(word):
    res = []
    for char in word:
        try:
            res.append(kanji_dictionary[char])
        finally:
            continue
    return res

def make_kanji_reading(kanjis):
    res = ""
    for kanji in kanjis:
        res += f"[{kanji.reading}] "
    return res

dictionary = collections.defaultdict(lambda:"")





katakana_start = 69432

kanji_start = 87297





In [167]:
#process kanji section



for entry in words[kanji_start:-1]:
    if ',' in entry:
        lhs,rhs = entry.split(',',maxsplit=1)
        fragments = lhs.replace("\t","").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)
        dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis,fragments[1],rhs)
        #print(f"{word_surface} {desc}")
    else:
        
        fragments = entry.replace('\t',"").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)

        try: 
            dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis, fragments[1],fragments[-1])
        except IndexError:
            print(fragments)
            continue





['揶揄うXem からかう']
['追放するXem 追放']


In [168]:


#process hiragana

for entry in words[0:katakana_start-1]:
    if ',' in entry:
        lhs,rhs = entry.split(',',maxsplit=1)
        fragments = lhs.replace("\t","").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)
        dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis,fragments[1],rhs)
        #print(f"{word_surface} {desc}")
    else:
        
        fragments = entry.replace('\t',"").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)

        try: 
            dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis, fragments[1],fragments[-1])
        except IndexError:
            print(fragments)
            continue


['あんざんXem 安産 @暗算 Xem 暗算']
['あんちゅうもさくXem 暗中模索']
['あんちょくXem 安直']
['こうひいXem 珈琲']
['こくどけいかくXem 国土計画']
['こくどこうつうしょうXem 国土交通省']
['ことばXem 言葉']
['ご馳走Xem ごちそう']
['しゃちょうXem 社長']
['じょうけんXem 条件']
['じんかいせんじゅつXem 人海戦術']
['せいてきXem 正嫡 Xem 清適 Xem 静的 Xem 政敵 Xem 性的']
['だんせいXem 弾性 Xem 男声 Xem 男性']
['だんせいホルモンXem 男性ホルモン']
['ちじょうせんXem 地上戦']
['ちゅうかんたいXem 中間体']
['ちゅうかんほうこくXem 中間報告']
['ちゅうかんテストXem 中間テスト']
['ちゅうしんぶXem 中心部']
['ちりがくXem 地理学']
['なんもんXem 難問 Xem 南門']
['のうそんXem 農村']
['のうどXem 農奴 Xem 濃度']
['のうみんXem 農民']
['のぞましいXem 望ましい']
['ぶっきょうとXem 仏教徒']
['まぼろしXem 幻']
['みかずきXem 三日月']
['みかづきXem 三日月']
['みつげつXem 蜜月']
['むずかしいXem 難しい']
['らちXem 拉致']
['れっしXem 烈士']
['ろうどうしゃXem 労働者 Xem 労動者']
['ろうどうじかんXem 労働時間']


In [169]:
#process katakana section
for entry in words[katakana_start:kanji_start-1]:
    if ',' in entry:
        lhs,rhs = entry.split(',',maxsplit=1)
        fragments = lhs.replace("\t","").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)
        dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis,fragments[1],rhs)
        #print(f"{word_surface} {desc}")
    else:
        
        fragments = entry.replace('\t',"").split('-')
        kanjis = look_up_kanji(fragments[0])
        kanji_reading = make_kanji_reading(kanjis)

        try: 
            dictionary[fragments[0]] = DictEntry(fragments[0],kanji_reading,kanjis, fragments[1],fragments[-1])
        except IndexError:
            print(fragments)
            continue


['コーヒーXem 珈琲']


In [170]:
dictionary["一回分"].kanji_reading

'[nhất] [hối/hồi] [phân/phần/phận] '

put into database

In [181]:
con = sqlite3.connect('dictionary.db')
cur = con.cursor()

In [182]:
cur.execute("DROP TABLE IF EXISTS kanjis ")
cur.execute("DROP TABLE IF EXISTS dict_entries ")
cur.execute("DROP TABLE IF EXISTS dict_entry_kanjis ")

In [183]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS kanjis (
    id INTEGER PRIMARY KEY,
    kanji TEXT NOT NULL,
    reading TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS dict_entries (
    id INTEGER PRIMARY KEY,
    word TEXT NOT NULL,
    kanji_reading TEXT,
    reading TEXT NOT NULL,
    desc TEXT
);

CREATE TABLE IF NOT EXISTS dict_entry_kanjis (
    dict_entry_id INTEGER NOT NULL,
    kanji_id INTEGER NOT NULL,
    PRIMARY KEY (dict_entry_id, kanji_id),
    FOREIGN KEY (dict_entry_id) REFERENCES dict_entries (id) ON DELETE CASCADE,
    FOREIGN KEY (kanji_id) REFERENCES kanjis (id) ON DELETE CASCADE
);
""")


In [ ]:
def insert_kanji(kanji: Kanji) -> int:
  
    try:
        # Check if the kanji already exists
        _ = cur.execute("SELECT id FROM kanjis WHERE kanji = ?", (kanji.kanji,))
        result = cur.fetchone()

        if result:
            kanji_id = result[0]
            #print(f"Kanji '{kanji.kanji}' already exists with ID: {kanji_id}")
        else:
            # Insert the new kanji
            _ = cur.execute(
                "INSERT INTO kanjis (kanji, reading) VALUES (?, ?)",
                (kanji.kanji, kanji.reading)
            )
            con.commit()
            kanji_id = cur.lastrowid
            #print(f"Inserted new kanji '{kanji.kanji}' with ID: {kanji_id}")
            
    except sqlite3.Error as e:
        print(f"Database error inserting kanji: {e}")
        con.rollback()
        kanji_id = -1
    finally:
        pass


def insert_dict_entry(entry: DictEntry) -> int:
    """
    Inserts a single DictEntry object and its associated Kanjis into the database.
    This handles the many-to-many relationship using the junction table.
    Returns the row ID of the new dict entry.
    """
    dict_entry_id = -1

    try:
        # Insert the DictEntry into the dict_entries table
        _ =cur.execute(
            "INSERT INTO dict_entries (word, kanji_reading, reading, desc) VALUES (?, ?, ?, ?)",
            (entry.word, entry.kanji_reading, entry.reading, entry.desc)
        )
        dict_entry_id = cur.lastrowid
        
        #print(f"Inserted dictionary entry for '{entry.word}' with ID: {dict_entry_id}")

        # Now, handle the list of kanjis and the junction table
        if entry.kanjis:
            #print("Processing associated kanjis...")
            for kanji_obj in entry.kanjis:
                # Use a separate cursor to check/insert each kanji to avoid issues with nested transactions
                kanji_cursor = con.cursor()
                kanji_cursor.execute("SELECT id FROM kanjis WHERE kanji = ?", (kanji_obj.kanji,))
                kanji_result = kanji_cursor.fetchone()

                if kanji_result:
                    kanji_id = kanji_result[0]
                else:
                    _ =kanji_cursor.execute(
                        "INSERT INTO kanjis (kanji, reading) VALUES (?, ?)",
                        (kanji_obj.kanji, kanji_obj.reading)
                    )
                    kanji_id = kanji_cursor.lastrowid
                
                # Insert the relationship into the junction table
                _ =cur.execute(
                    "INSERT INTO dict_entry_kanjis (dict_entry_id, kanji_id) VALUES (?, ?)",
                    (dict_entry_id, kanji_id)
                )
            
        con.commit()
        #print("All related data committed successfully.")
            
    except sqlite3.IntegrityError as e:
        print(f"Integrity Error: {e}. The entry might already exist.")
        con.rollback()
        dict_entry_id = -1
    except sqlite3.Error as e:
        print(f"Database error inserting dict entry: {e}")
        con.rollback()
        dict_entry_id = -1
    finally:
        pass

In [178]:
for kanji in kanji_dictionary.values():
    insert_kanji(kanji)

Inserted new kanji '報' with ID: 2527
Inserted new kanji '奉' with ID: 2528
Inserted new kanji '宝' with ID: 2529
Inserted new kanji '峰' with ID: 2530
Inserted new kanji '峯' with ID: 2531
Inserted new kanji '崩' with ID: 2532
Inserted new kanji '庖' with ID: 2533
Inserted new kanji '抱' with ID: 2534
Inserted new kanji '捧' with ID: 2535
Inserted new kanji '放' with ID: 2536
Inserted new kanji '方' with ID: 2537
Inserted new kanji '朋' with ID: 2538
Inserted new kanji '法' with ID: 2539
Inserted new kanji '泡' with ID: 2540
Inserted new kanji '烹' with ID: 2541
Inserted new kanji '砲' with ID: 2542
Inserted new kanji '縫' with ID: 2543
Inserted new kanji '胞' with ID: 2544
Inserted new kanji '芳' with ID: 2545
Inserted new kanji '萌' with ID: 2546
Inserted new kanji '蓬' with ID: 2547
Inserted new kanji '蜂' with ID: 2548
Inserted new kanji '褒' with ID: 2549
Inserted new kanji '訪' with ID: 2550
Inserted new kanji '豊' with ID: 2551
Inserted new kanji '邦' with ID: 2552
Inserted new kanji '鋒' with ID: 2553
I

KeyboardInterrupt: 

In [153]:
for entry in dictionary.values():
    insert_dict_entry(entry)

Integrity Error: UNIQUE constraint failed: dict_entry_kanjis.dict_entry_id, dict_entry_kanjis.kanji_id. The entry might already exist.
Integrity Error: UNIQUE constraint failed: dict_entry_kanjis.dict_entry_id, dict_entry_kanjis.kanji_id. The entry might already exist.
Integrity Error: UNIQUE constraint failed: dict_entry_kanjis.dict_entry_id, dict_entry_kanjis.kanji_id. The entry might already exist.
Integrity Error: UNIQUE constraint failed: dict_entry_kanjis.dict_entry_id, dict_entry_kanjis.kanji_id. The entry might already exist.


KeyboardInterrupt: 